In [46]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import random

from torch import nn
import torch.optim as optim

In [47]:
class WasteDataset(Dataset):
    def __init__(self, root_dir, split, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.image_paths = []
        self.labels = []
        
        for i, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for subfolder in ['default', 'real_world']:
                subfolder_dir = os.path.join(class_dir, subfolder)
                image_names = os.listdir(subfolder_dir)
                random.shuffle(image_names)
                
                if split == 'train':
                    image_names = image_names[:int(0.6 * len(image_names))]
                elif split == 'val':
                    image_names = image_names[int(0.6 * len(image_names)):int(0.8 * len(image_names))]
                else:  # split == 'test'
                    image_names = image_names[int(0.8 * len(image_names)):]
                
                for image_name in image_names:
                    self.image_paths.append(os.path.join(subfolder_dir, image_name))
                    self.labels.append(i)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
# Child class inheretied from parent class (nn.Module: Base class for all neural network modules)
class Custom_CNN_Model(nn.Module):
    def __init__(self, waste_classes_nb):
        super().__init__()
        self.conv1 = nn.Conv2d(3,out_channels=16, kernel_size=4,stride=1,padding=1)
        self.conv2 = nn.Conv2d(16,out_channels=32, kernel_size=3,stride=1,padding=1)
        self.conv3 = nn.Conv2d(32,out_channels=64, kernel_size=3,stride=1,padding=1)
        self.conv4 = nn.Conv2d(64,out_channels=128, kernel_size=3,stride=1,padding=1)
        self.conv5 = nn.Conv2d(128,out_channels=256, kernel_size=3,stride=1,padding=1)
        self.conv6 = nn.Conv2d(256,out_channels=512, kernel_size=3,stride=1,padding=1)

        self.relu = nn.ReLU()
        self.maxpooling = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(512*6*6, 512)
        self.fc2 = nn.Linear(512, waste_classes_nb)


    def forward(self, mod):
        mod = self.conv1(mod)  # 16 x 223 x 223
        mod = self.relu(mod)
        mod = self.maxpooling(mod)  # 16 x 111 x 111
        
        mod = self.conv2(mod) # 32 x 111 x 111
        mod = self.relu(mod)
        mod = self.maxpooling(mod) # 32 x 55 x 55

        mod = self.conv3(mod)  # 64 x 55 x 55
        mod = self.relu(mod)
        mod = self.conv4(mod)  # 128 x 55 x 55
        mod = self.relu(mod)
        mod = self.maxpooling(mod) # 128 x 27 x 27

        mod = self.conv5(mod) # 256 x 27 x 27
        mod = self.relu(mod)
        mod = self.maxpooling(mod) # 256 x 13 x 13

        mod = self.conv6(mod) # 512 x 13 x 13
        mod = self.relu(mod)
        mod = self.maxpooling(mod) # 512 x 6 x 6
        

        mod = mod.view(mod.size(0), -1)

        mod = self.fc1(mod)
        mod = self.relu(mod)
        mod = self.fc2(mod)
        return mod


In [49]:
# Load dataset path
dataset_path = r'C:\Users\rouka\OneDrive\Desktop\Uni\year 3\semester 2\CSE 351 - Intro to AI\Final_Project\archive\images\images'

# Set dataset hyperparameters
batch_size = 32
num_epochs = 5
learning_rate = 0.001

In [50]:
# Create the datasets and data loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = WasteDataset(dataset_path, split='train', transform=transform)
val_dataset = WasteDataset(dataset_path, split='val', transform=transform)
test_dataset = WasteDataset(dataset_path, split='test', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [51]:
Waste_Classes = train_dataset.classes
print(Waste_Classes)

Waste_Classes_NB = len(Waste_Classes)
print(Waste_Classes_NB)

['shoes', 'steel_food_cans', 'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags']
5


In [52]:
model = Custom_CNN_Model(Waste_Classes_NB)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
# Lists to store the training and validation losses
train_losses = []
val_losses = []

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for images, labels in train_dataloader:
        images = images
        labels = labels
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
    
    train_loss /= len(train_dataset)
    train_losses.append(train_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}")

print("Training completed!")

Epoch [1/5], Train Loss: 1.4801
Epoch [2/5], Train Loss: 1.2973
Epoch [3/5], Train Loss: 1.2022
Epoch [4/5], Train Loss: 1.1075
Epoch [5/5], Train Loss: 1.0374
Training completed!
